[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    1. Initialization
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Initialize all the librairies needed for the project;</li>

In [4]:
# -- IMPORTATION OF THE PACKAGES --
import XLSX, DataFrames
using JuMP, Gurobi, Printf, Dates

In [5]:
# -- FUNCTION TO PRINT TITLE --

function print_title(title::String)
    println(title)
    for i in 1:length(title) @printf("-") end
    @printf("\n")
end

print_title (generic function with 1 method)

In [6]:
# -- FUNCTION TO PRINT 1D ARRAY --

function print_vector(vector::Vector{<:Number}, units)
    println("["*join(vector, ", ")*"] [$units]")
end

print_vector (generic function with 1 method)

[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    2. Definition of the network model 
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Load a network model from an Excel file;</li>
        <li style="margin-bottom:10px">Initialize the parameters defining the network</li>
    </ul> 

In [7]:
# -- LOADING OF THE EXCEL FILE --

XLSX_FILE_PATH = joinpath(splitdir(@__DIR__)[1], "network_models/network_Nahman_Peric_2S23H.xlsx")

"/Users/manoncornet/Documents/University/TFE/DNEP/network_models/network_Nahman_Peric_2S23H.xlsx"

In [9]:
# -- DEFINITION OF THE PER UNIT BASIS --
# Values obtained by reading the Paper "A Constructive Heuristic Algorithm for Distribution System Planning"

const BASE_VOLTAGE    = 34.5                             # [kV]
const BASE_POWER      = 1                                # [MVA]
const BASE_CURRENT    = BASE_POWER / BASE_VOLTAGE        # [kA]
const BASE_ADMITTANCE = BASE_CURRENT / BASE_VOLTAGE      # [S]
const BASE_IMPEDANCE  = 1/BASE_ADMITTANCE                # [Ohm]
print_title("Per-unit basis:")

@printf("Base Admittance : %.3e [S]\n", BASE_ADMITTANCE)
@printf("Base Impedance  : %.3e [Ohm]\n", BASE_IMPEDANCE)
@printf("Base Voltage    : %.3e [kV]\n", BASE_VOLTAGE)
@printf("Base Current    : %.3e [kA]\n", BASE_CURRENT)
@printf("Base Power      : %.3e [MVA]\n", BASE_POWER)

Per-unit basis:
---------------
Base Admittance : 8.402e-04 [S]
Base Impedance  : 1.190e+03 [Ohm]
Base Voltage    : 3.450e+01 [kV]
Base Current    : 2.899e-02 [kA]
Base Power      : 1.000e+00 [MVA]


In [10]:
# -- FUNCTION THAT PROCESSES THE LINE PROPRETIES  --

function process_conductors(df_cond::DataFrames.DataFrame, 
                            len_lines::Vector{Float64},  
                            nb_lines::Integer, 
                        )

    nb_cond = DataFrames.nrow(df_cond)
    max_i   = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    r       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    x       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    g       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    b       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    
    line_cost = Array{Float64}(undef, nb_lines, nb_cond) # [€/km]

    # Only take the first conductors of the list in the file
    for l in 1:nb_lines
        for k in 1:nb_cond
            max_i[l, k] = df_cond.max_i_ka[k] ./ BASE_CURRENT

            r[l, k] = len_lines[l] * df_cond.r_ohm_per_km[k] ./ BASE_IMPEDANCE
            x[l, k] = len_lines[l] * df_cond.x_ohm_per_km[k] ./ BASE_IMPEDANCE
            y = 1/(r[l, k]+im*x[l, k])

            g[l, k] = real(y) 
            b[l, k] = abs(imag(y))

            line_cost[l, k] = df_cond.cost_kdollars_per_km[k] * len_lines[l]
        end
    end
    return max_i, line_cost, r, x, g, b
end

process_conductors (generic function with 1 method)

In [11]:
# -- FETCH THE DATA FROM THE EXCEL SHEET --

df_bus  = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "bus"))
df_line = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "line"))
df_cond = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "conductor"))

Row,idx,Name,d_mm,q_mm2,r_ohm_per_km,x_ohm_per_km,max_i_ka,cost_kdollars_per_km
,Any,Any,Any,Any,Any,Any,Any,Any
1,1,AAC (1/0),9.35,53.5,0.5502,0.429,0.23,10
2,2,AAC (4/0),13.26,107.3,0.2707,0.402,0.34,20


In [12]:
# -- LINE PARAMETERS DEFINITION --

L_size = DataFrames.nrow(df_line)                                   # Number of lines in the network
L      = 1:L_size                                                   # Line set
line_ends = [(df_line.from_bus[l], df_line.to_bus[l]) for l in L]   # Indices of the line extremities
len_lines = convert(Vector{Float64}, df_line.length_km)             # Line lengths [km]

print_title("Network lines summary:")
@printf("Number of lines : %d\n", L_size)

Network lines summary:
----------------------
Number of lines : 34


In [13]:
# -- DEFINITION OF THE PHYSICAL QUANTITIES ASSOCIATED TO NETWORK LINES --

K_size = DataFrames.nrow(df_cond)   # Number of conductor types
K      = 1:K_size                   # Set of conductors

max_i, line_cost, R, X, G, B, = process_conductors(df_cond, len_lines, L_size)

print_title("Conductor properties:")
println(df_cond)

Conductor properties:
---------------------
2×8 DataFrame
 Row │ idx  Name       d_mm   q_mm2  r_ohm_per_km  x_ohm_per_km  max_i_ka  cost_kdollars_per_km 
     │ Any  Any        Any    Any    Any           Any           Any       Any                  
─────┼──────────────────────────────────────────────────────────────────────────────────────────
   1 │ 1    AAC (1/0)  9.35   53.5   0.5502        0.429         0.23      10
   2 │ 2    AAC (4/0)  13.26  107.3  0.2707        0.402         0.34      20


In [14]:
# -- BUS PARAMETERS DEFINITION --

N_size = DataFrames.nrow(df_bus)            # Number of buses in the network
N      = 1:N_size                           # Buses set

Ns_size = sum(df_bus.type .== "substation") # Number of substation buses
Nu_size = sum(df_bus.type .== "user")       # Number of load nodes

Ns_init_size = sum(df_bus.S_G_init_mva .> 0)
Ns = 1:Ns_size                              # Set of substation buses
Nu = (1:Nu_size) .+ Ns_size                 # Set of load buses
Ns_init = 1:Ns_init_size

print_title("Network buses summary:")
@printf("Number of buses            : %d\n", N_size)
@printf("Number of load buses       : %d\n", Nu_size)
@printf("Number of existing substation buses : %d\n", Ns_init_size)
@printf("Number of substation buses : %d\n", Ns_size)

Network buses summary:
----------------------
Number of buses            : 23
Number of load buses       : 21
Number of existing substation buses : 1
Number of substation buses : 2


In [15]:
# -- DEFINITION OF THE PHYSICAL QUANTITIES ASSOCIATED TO NETWORK BUSES --

# Limits on voltage
MIN_VOLTAGE = 0.97  # [pu]
MAX_VOLTAGE = 1.03  # [pu]

# Demand at buses
# Assumption: load power factor is constant for all loads and is lagging (inductive)
cos_phi = 0.9
S_D     = convert(Vector{Float64}, df_bus.S_D_mva) ./ BASE_POWER
P_D     = S_D * cos_phi
Q_D     = S_D * sin(acos(cos_phi))

print_title("Network buses additional info:")
@printf("Power factor  : %.3g\n", cos_phi)
@printf("Voltage range : [%1.3g, %1.3g] [pu]", MIN_VOLTAGE, MAX_VOLTAGE)

Network buses additional info:
------------------------------
Power factor  : 0.9
Voltage range : [0.97, 1.03] [pu]

In [16]:
# -- SUBSTATION PARAMETERS DEFINITION --

S_rating_init    = convert(Vector{Float64}, df_bus.S_G_init_mva[Ns]) ./ BASE_POWER # [pu]
S_rating_max     = convert(Vector{Float64}, df_bus.S_G_max_mva[Ns]) ./ BASE_POWER # [pu]
sub_install_cost = 1e3      # k$
sub_op_cost      = 0.1*1e-3 # k$/kVah^2

print_title("Substation buses additional info:")
print("Initial rating of substations: "); print_vector(S_rating_init, "pu");
print("Maximum rating of substations: "); print_vector(S_rating_max, "pu");
@printf("Substation construction cost : %.3g [k\$]\n", sub_install_cost)
@printf("Substation operation cost    : %.3g [k\$/kVA^2]", sub_op_cost)

Substation buses additional info:
---------------------------------
Initial rating of substations: [4.0, 0.0] [pu]
Maximum rating of substations: [4.0, 4.0] [pu]
Substation construction cost : 1e+03 [k$]
Substation operation cost    : 0.0001 [k$/kVA^2]

In [17]:
# -- LINK BTW LINES AND NODES --

Omega_sending   = Dict(n => [] for n in N)
Omega_receiving = Dict(n => [] for n in N)
for l in L
    push!(Omega_sending[line_ends[l][1]], l)
    push!(Omega_receiving[line_ends[l][2]], l)
end

print_title("Link btw lines and nodes:")
println("Omega sending  : $Omega_sending")
println("Omega receiving: $Omega_receiving")

Link btw lines and nodes:
-------------------------
Omega sending  : Dict{Int64, Vector{Any}}(5 => [21, 22], 16 => [], 20 => [], 12 => [18], 8 => [28, 29, 30, 33], 17 => [17], 1 => [1], 19 => [5, 6, 7], 22 => [8, 9], 23 => [19, 20], 6 => [25, 26], 11 => [10, 11], 9 => [32], 14 => [24], 3 => [34], 7 => [27], 4 => [23, 31], 13 => [12], 15 => [13, 14, 15], 2 => [], 10 => [2, 3, 4], 18 => [16], 21 => [])
Omega receiving: Dict{Int64, Vector{Any}}(5 => [20], 16 => [8, 25, 34], 20 => [2, 5], 12 => [15], 8 => [27], 17 => [16], 1 => [], 19 => [4], 22 => [7], 23 => [18], 6 => [23, 24], 11 => [9], 9 => [29, 31], 14 => [3, 17, 19, 21], 3 => [32, 33], 7 => [26], 4 => [22, 30], 13 => [11], 15 => [12], 2 => [28], 10 => [1], 18 => [14], 21 => [6, 10, 13])


[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    3. Definition of the objective function parameters 
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Set the parameters that are required in the objective function;</li>
    </ul> 

In [18]:
# -- FUNCTION FOR COMPUTING THE NPV --

function PV_coeff(tau, lambda)
    return (1 - 1/(1 + tau)^lambda)/tau  
end

# Capital recovery rate formula
# tau: interest rate 
# n : number of annuity received
function CRF(tau, n)
    return (tau * (1 + tau)^n)/((1 + tau)^n - 1)  
end

CRF (generic function with 1 method)

In [19]:
# -- OBJECTIVE FUNCTION PARAMETERS --

nb_years_planning = 1
delta_t           = 1 # [h]

tau = 0.1

line_loss_factor = 0.35     # phi_l : loss factor of lines
sub_loss_factor  = 0.35     # phi_s : cost per energy lost [€/kWh]

K_l = CRF(tau, 1)           # Capital recovery rate of line constructions
K_s = CRF(tau, 1)           # Capital recovery rate of substation construction or reinforcement

loss_cost = 0.05*1e-3       # [k$/kWh]
tau_l     = tau             # tau_l : interest rate of circuits
tau_s     = tau             # tau_s : interest rate of substations

f_l = PV_coeff(tau_l, nb_years_planning)
f_s = PV_coeff(tau_s, nb_years_planning)

0.9090909090909094

In [20]:
# -- CREATION OF THE NETWORK DICT --
# Rmq: all the costs are in k$
Ns_init = [1]
network_dict = Dict(:bus       => (N, Omega_sending, Omega_receiving, MIN_VOLTAGE, MAX_VOLTAGE),
                    :load_bus  => (Nu, P_D, Q_D, delta_t),
                    :sub_bus   => (Ns, Ns_init, S_rating_init, S_rating_max),
                    :line      => (L, line_ends, max_i, R, X, G, B),
                    :conductor => (K)
                    )
                    
obj_fct_dict = Dict(:LF  => (line_loss_factor, sub_loss_factor),
                    :CRF => (K_l, K_s),
                    :NPV_coeff => (f_l, f_s),
                    :costs => (loss_cost, sub_install_cost, sub_op_cost, line_cost)
                    )

Dict{Symbol, Tuple{Float64, Float64, Vararg{Any}}} with 4 entries:
  :NPV_coeff => (0.909091, 0.909091)
  :LF        => (0.35, 0.35)
  :CRF       => (1.1, 1.1)
  :costs     => (5.0e-5, 1000.0, 0.0001, [2.02092 4.04183; 6.97278 13.9456; … ;…

In [29]:
# -- CONFIGURATION STRUCTURE --
# It becomes very very hard to code => Think of other ways
abstract type PowerFlowFormulation end      # can be either Jabr or distflow
abstract type TypeofProdFormulation end     # No_DG or DG generation (boolean) => replace by a "if" 
abstract type NetworkGraphFormulation end   # directed or undirected (boolean) => complicated to replace by a "if"
abstract type RadialityFormulation end      # simple - single commodity flow - multi-commodity - spanning tree constraints
abstract type CondVarsFormulation end

struct BIM <: PowerFlowFormulation end
struct BFM <: PowerFlowFormulation end

struct NoDG <: TypeofProdFormulation end
struct DG <: TypeofProdFormulation end

struct DirectedGraph <: NetworkGraphFormulation end
struct UnDirectedGraph <: NetworkGraphFormulation end

struct SimpleRadiality <: RadialityFormulation end
struct SingleCommodityFlow <: RadialityFormulation end

# Choice to aggregate or not with a variable the values of branch variables over all conductors
struct AggrCondVars <: CondVarsFormulation end 
struct NonAggrCondVars <: CondVarsFormulation end

struct Formulation
    powerflow::PowerFlowFormulation 
    production::TypeofProdFormulation
    radiality::RadialityFormulation
    networkgraph::NetworkGraphFormulation
    condvars::CondVarsFormulation

    function Formulation(;
        powerflow::PowerFlowFormulation = BFM(),
        production::TypeofProdFormulation = NoDG(),
        radiality::RadialityFormulation = SimpleRadiality(),  
        networkgraph::NetworkGraphFormulation = DirectedGraph(),
        condvars::CondVarsFormulation = NonAggrCondVars()           
        )                     
        return new(powerflow, production, radiality, networkgraph, condvars)
    end
end

In [30]:
# -- BUS VARIABLES --
 
function _add_BusVariables!(model, ::NoDG)::Nothing 

    N   = model[:network_data][:bus][1]
    Ns  = model[:network_data][:sub_bus][1]
    MIN_VOLTAGE = model[:network_data][:bus][4]
    MAX_VOLTAGE = model[:network_data][:bus][5]

    @variables( model,   
                begin 
                MIN_VOLTAGE^2 <= V_sqr[N] <= MAX_VOLTAGE^2 , (container=Array)
                P_G[N] >= 0                                , (container=Array)
                Q_G[N]                                     , (container=Array)
                S_G[N]                                     , (container=Array)
                beta[Ns]                                   , (container=Array, binary=true)    
                end
            )
            
    for i in Nu
        fix(P_G[i], 0.0; force=true) 
        fix(Q_G[i], 0.0)
        fix(S_G[i], 0.0)
    end
    
    return
end

function _add_BusVariables!(model, ::DG)::Nothing 

    N   = model[:network_data][:bus][1]
    Ns  = model[:network_data][:sub_bus][1]
    MIN_VOLTAGE = model[:network_data][:bus][4]
    MAX_VOLTAGE = model[:network_data][:bus][5]

    @variables( model,   
                begin 
                MIN_VOLTAGE^2 <= V_sqr[N] <= MAX_VOLTAGE^2 , (container=Array)
                P_G[N] >= 0                                , (container=Array)
                Q_G[N]                                     , (container=Array)
                S_G[N]                                     , (container=Array)
                beta[Ns]                                   , (container=Array, binary=true)    
                end
            )
    return
end

_add_BusVariables! (generic function with 2 methods)

In [38]:
# -- BRANCH VARIABLES --
# Here maybe think to aggregate or not in variables, another option 
function _add_BranchVariables!(model, ::DirectedGraph, ::NonAggrCondVars)::Nothing 
    # Here maybe try to test another formulations without variables
    L = model[:network_data][:line][1]
    K = model[:network_data][:conductor]

    JuMP.@variables( model,   
                begin 
                P_ij_k[L, K]        , (container=Array) 
                Q_ij_k[L, K]        , (container=Array)
                I_sqr_k[L, K] >= 0  , (container=Array)
                end
    )

    return
end

function _add_BranchVariables!(model, ::UnDirectedGraph, ::NonAggrCondVars)::Nothing 
    # Here maybe try to test another formulations without variables
    L   = model[:network_data][:line][1]
    K   = model[:network_data][:conductor]

    JuMP.@variables( model,   
                begin 
                P_ij_k[L, K]        , (container=Array) 
                P_ji_k[L, K]        , (container=Array) 
                Q_ij_k[L, K]        , (container=Array)
                Q_ji_k[L, K]        , (container=Array)
                I_sqr_k[L, K] >= 0  , (container=Array)
                end
    )
    return
end

function _add_BranchVariables!(model, ::DirectedGraph, ::AggrCondVars)::Nothing 
    # Here maybe try to test another formulations without variables
    L   = model[:network_data][:line][1]
    K   = model[:network_data][:conductor]

    JuMP.@variables( model,   
                begin 
                P_ij_k[L, K]        , (container=Array) 
                Q_ij_k[L, K]        , (container=Array)
                I_sqr_k[L, K] >= 0  , (container=Array)
                P_ij[L]             , (container=Array) 
                Q_ij[L]             , (container=Array) 
                I_sqr[L] >= 0       , (container=Array)
                end
    )

    return
end

function _add_BranchVariables!(model, ::UnDirectedGraph, ::AggrCondVars)::Nothing 
    # Here maybe try to test another formulations without variables
    L   = model[:network_data][:line][1]
    K   = model[:network_data][:conductor]

    JuMP.@variables( model,   
                begin 
                P_ij_k[L, K]        , (container=Array) 
                P_ji_k[L, K]        , (container=Array) 
                Q_ij_k[L, K]        , (container=Array)
                Q_ji_k[L, K]        , (container=Array)
                I_sqr_k[L, K] >= 0  , (container=Array)
                P_ij[L]             , (container=Array)
                P_ji[L]             , (container=Array) 
                Q_ij[L]             , (container=Array)
                Q_ji[L]             , (container=Array)  
                I_sqr[L] >= 0       , (container=Array)
                end
    )

    return
end

_add_BranchVariables! (generic function with 4 methods)

In [32]:
# -- REFERENCE VOLTAGE AT SUBSTATIONS -- 

function _add_RefVoltages!(model)::Nothing

    Ns      = model[:network_data][:sub_bus][1]
    Ns_init = model[:network_data][:sub_bus][2]
    Ns_notinit = setdiff(Ns, Ns_init)
    MIN_VOLTAGE = model[:network_data][:bus][4]
    MAX_VOLTAGE = model[:network_data][:bus][5]

    JuMP.@constraint(model, 
                ref_voltage_sub_init[i=Ns_init], 
                model[:V_sqr][i] == 1
    )
    
    JuMP.@constraint(model, 
                ref_voltage_sub_notinit1[i=Ns_notinit], 
                model[:V_sqr][i] - 1 <= (MAX_VOLTAGE^2 - 1)*(1 - model[:beta][i])
    )

    JuMP.@constraint(model, 
                ref_voltage_sub_notinit2[i=Ns_notinit], 
                model[:V_sqr][i] - 1 >= (MIN_VOLTAGE^2 - 1)*(1 - model[:beta][i])
    ) 
    
    return 
end

_add_RefVoltages! (generic function with 1 method)

In [33]:
# -- LOAD OVERSATISFACTION CONSTRAINTS --

function _add_LoadOverSatisfaction!(model)::Nothing

    N   = model[:network_data][:bus][1]
    P_D = model[:network_data][:load_bus][2]
    Q_D = model[:network_data][:load_bus][3]
    
    JuMP.@constraint(model, 
                    LoadOverSatisfaction_active, 
                    sum(model[:P_G][i] - P_D[i] for i in N) >= 0
    )

    JuMP.@constraint(model, 
                    LoadOverSatisfaction_reactive, 
                    sum(model[:Q_G][i] - Q_D[i] for i in N) >= 0
    )

    return 
end

_add_LoadOverSatisfaction! (generic function with 1 method)

In [39]:
# Function that builds the model
function build_model(network_dict, obj_dict;
    formulation = Formulation()
    )::JuMP.Model

    @info "Building model..."
    time_model = @elapsed begin
        model = JuMP.Model(Gurobi.Optimizer)
        set_optimizer_attribute(model, "TimeLimit", 200)
        set_optimizer_attribute(model, "Presolve", 0)

        model[:network_data] = network_dict
        model[:obj_data]     = obj_dict

        _add_BusVariables!(model, formulation.production)
        _add_BranchVariables!(model, formulation.networkgraph, formulation.condvars)
        _add_RefVoltages!(model)
        _add_LoadOverSatisfaction!(model)
        _add_PowerFlowEqs!(model, formulation.powerflow, formulation.networkgraph, formulation.condvars)
            
    end

    @info @sprintf("Built model in %.2f seconds", time_model)


    return model
end

build_model (generic function with 1 method)

In [40]:
model = build_model(network_dict, obj_fct_dict)
print(model)

[ Info: Building model...
[ Info: Built model in 1.23 seconds


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 200
Set parameter Presolve to value 0
